In [1]:
#загрузим библиотеку и посмотрим на текста:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [3]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
n=1249
print(newsgroups_train.target_names[newsgroups_train.target[n]])
print(newsgroups_train.data[n])

sci.med

I have a new doctor who gave me a prescription today for something called 
Septra DS.  He said it may cause GI problems and I have a sensitive stomach 
to begin with.  Anybody ever taken this antibiotic.  Any good?  Suggestions 
for avoiding an upset stomach?  Other tips?



In [15]:
#загрузим векторайзер
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True,min_df=10, max_df=.04)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.04, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'sometime', 'was', 'would', 'third', 'cannot', 'bottom', 'each', 'everyone', 'wherein', 'hereupon', 'almost', 'some', 'twenty', 'against', 'off', 'became', 'thin', 'moreover', 'our', 'because', 'neither', 'once', 'herself', 'less', 'now', 'noone', 'under', 'in', 'not', 'please'... 'than', 'couldnt', 'forty', 'beforehand', 'ourselves', 'have', 'well', 'above', 'whereas', 'call'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
vectorizer.vocabulary_.get('window')

10091

In [17]:
text = 'I was wondering if anyone out there could enlighten me on this car I saw'
x = vectorizer.transform([text])
vectorizer.inverse_transform(x)

[array(['car', 'enlighten', 'saw', 'wondering'], dtype='<U79')]

In [18]:
x.data

array([1, 1, 1, 1], dtype=int64)

In [19]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 10299)

In [23]:

from tqdm import tqdm

def lda (X_train, K, alpha, beta, n_iter):
    n_k_w = np.zeros((K, X_train.shape[1])) #заводим cчетчики
    n_d_k = np.zeros((X_train.shape[0], K))
    n_k = np.zeros(K)
    
    doc, word = X_train.nonzero()
    tags = np.random.choice(K, len(doc)) #присваиваем всем словам случайные теги
    
    for w,d,t in zip(word, doc, tags):
        n_k_w[t,w] += 1
        n_d_k[d,t] += 1
        n_k[t] +=1
        
        
    for i in tqdm(range(n_iter)): #будем выполнять алгоритм n_iter раз (пока не сойдется к стационарному)
        for j in range(len(doc)):
            t = tags[j]
            n_k_w[t,word[j]] -= 1
            n_d_k[doc[j], t] -=1
            n_k[t] -=1
            
            p = (n_d_k[doc[j], :] + alpha)*(n_k_w[:,word[j]] + beta[word[j]])/(n_k + beta.sum())
            tags[j] = np.random.choice(np.arange(K), p = p / p.sum())
            
            n_k_w[tags[j], word[j]] += 1
            n_d_k[ doc[j], tags[j]] += 1
            n_k[tags[j]] += 1
            
    return n_k_w, n_d_k, n_k, tags
            


In [24]:
K = 20
n_k_w, n_d_k, n_k, tags = lda(X_train, K, np.ones(K), np.ones(X_train.shape[1]),50)

100%|██████████████████████████████████████████| 50/50 [32:21<00:00, 41.80s/it]


In [26]:
top_words = np.argsort(n_k_w, axis=1)[:, :-11:-1]
for i in range(20):
    doc = np.zeros((1, X_train.shape[1]))
    for j in top_words[i]:
        doc[0, j] = 1
    print('Topic №{}:\t{}'.format(i, '\t'.join(vectorizer.inverse_transform(doc)[0])))

Topic №0:	00	100	50	condition	offer	original	price	sale	sell	shipping
Topic №1:	answer	control	couldn	folks	knows	newsgroup	reading	sounds	tom	willing
Topic №2:	1993	address	center	contact	date	general	nasa	research	space	university
Topic №3:	advance	card	computer	disk	dos	hi	mac	memory	pc	video
Topic №4:	country	crime	federal	gun	guns	law	laws	police	rights	self
Topic №5:	bible	christ	christian	christians	church	faith	jesus	love	man	religion
Topic №6:	clinton	days	mentioned	money	spend	tax	thank	told	wasn	week
Topic №7:	chip	clipper	encryption	key	keys	nsa	phone	public	secure	security
Topic №8:	11	12	13	14	16	17	18	19	24	25
Topic №9:	air	anybody	build	circuit	current	ground	heat	ones	small	sorry
Topic №10:	agree	argument	claim	discussion	evidence	feel	important	making	matter	mind
Topic №11:	children	israel	israeli	jews	killed	land	left	men	war	women
Topic №12:	game	games	hockey	league	play	players	season	team	teams	win
Topic №13:	anybody	appreciate	couple	especially	guy	hear	man	news	

По топ словам можно понять некоторые топики: 2) космос 5) религия 6) talk.politics.guns 11) мидл ист 12)хоккей 15)машины 
естественно, при увеличении количества итераций и размера словаря - получили бы более точную характеристику топиков. Но для этого требуется больше времени
